In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
(train_image, train_labels), (validation_image,validation_labels) = tfds.load('tf_flowers', split=['train[:70%]', 'train[:30%]'], as_supervised = True, batch_size = -1)

In [ ]:
train_image = tf.image.resize(train_image, (150, 150))
validation_image = tf.image.resize(validation_image, (150, 150))

In [ ]:
train_labels

<tf.Tensor: shape=(2569,), dtype=int64, numpy=array([2, 3, 3, ..., 0, 2, 0])>

In [ ]:
num_classes = 5
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
validation_labels = tf.keras.utils.to_categorical(validation_labels, num_classes)

In [ ]:
train_labels[0]

array([0., 0., 1., 0., 0.], dtype=float32)

## **Image Preprocessing**

In [ ]:
# Image Data Generator for data augmentation and standardization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True)

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow(train_image, train_labels, batch_size=32)
validation_generator = validation_datagen.flow(validation_image, validation_labels, batch_size=32)

## **Use Pretrained VGG16 Image Classification model**

In [ ]:
train_image[0].shape

TensorShape([150, 150, 3])

In [ ]:
base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_image[0].shape)

58889256/58889256 [==============================] - 4s 0us/step


In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [ ]:
# Add new fully-connected layers on top

model = Sequential()
model.add(base_model)

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))  # Dropout layer to prevent overfitting

model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))  # Output layer with 5 neurons for 5 flower classes

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(patience=5, monitor='val_loss')
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=3, monitor='val_loss')
callbacks = [early_stopping, reduce_lr]

In [ ]:
history = model.fit(train_generator, epochs=20, validation_data=validation_generator, callbacks=callbacks)

Epoch 1/20
81/81 [==============================] - 32s 291ms/step - loss: 1.6491 - accuracy: 0.2768 - val_loss: 1.3796 - val_accuracy: 0.4941 - lr: 0.0010
Epoch 2/20
81/81 [==============================] - 18s 221ms/step - loss: 1.4318 - accuracy: 0.3784 - val_loss: 1.2252 - val_accuracy: 0.5359 - lr: 0.0010
Epoch 3/20
81/81 [==============================] - 17s 215ms/step - loss: 1.3635 - accuracy: 0.4114 - val_loss: 1.1843 - val_accuracy: 0.5695 - lr: 0.0010
Epoch 4/20
81/81 [==============================] - 19s 232ms/step - loss: 1.2886 - accuracy: 0.4461 - val_loss: 1.1379 - val_accuracy: 0.5250 - lr: 0.0010
Epoch 5/20
81/81 [==============================] - 18s 228ms/step - loss: 1.2637 - accuracy: 0.4574 - val_loss: 1.0061 - val_accuracy: 0.6312 - lr: 0.0010
Epoch 6/20
81/81 [==============================] - 18s 220ms/step - loss: 1.1987 - accuracy: 0.5045 - val_loss: 0.9610 - val_accuracy: 0.6966 - lr: 0.0010
Epoch 7/20
81/81 [==============================] - 18s 228ms/st

In [ ]:
test_loss, test_acc = model.evaluate(validation_generator)
print('Test accuracy:', test_acc)

35/35 [==============================] - 3s 71ms/step - loss: 0.6561 - accuracy: 0.7684
Test accuracy: 0.7683923840522766


In [ ]:
model.save('tf_flowers_vgg16_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
